This notebook acquires data to measure the gain of the SiPM. It sets the overvoltage and then takes scope and MCA data with the Red Pitaya. The scope is setup to trigger on the LED.

In [1]:
import numpy as np
import time,datetime
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline
plt.style.use('../../style.mplstyle')
import sys 
sys.path.append("../../devices")
sys.path.append("../../RedPitaya")
from datetime import datetime

In [2]:
#### set data taking parameters here #####

save_dir = "" ## name of director to save, leave empty to use todays date as default
make_plots = True ## make diagnostic plots

breakdown_voltage = 27.38 ## Volts, breakdown to sweep keithley relative to
ov_list = np.arange(2, 7, 0.5) ## Volts, list of overvoltages to sweep

instr_to_sweep = 'synth' ## instrument to sweep over (must be 'keith' or 'synth')

red_pitaya_channel = 2 ## channel for the analog signal in the RP (1 or 2), used for MCA presumably after shaper
wf_chan_list = [1,2] ## list of RP channels to acquire waveforms for

num_scope_files=10 ## number of files to save the scope traces in
num_scope_traces = 200 ## number of traces at each overvoltage
num_mca_events = 1000 ## number of MCA pulses in the histogram
LED_flashing_rate = 100 ## Hz, rate at which to pulse LED

In [3]:
## setup directories for saving data

date = datetime.today().strftime('%Y%m%d')
subdir = ''
path = "C:/Data/{}/{}/".format(date,subdir)
if not os.path.exists(path):
    os.makedirs(path)

In [4]:
## import instrument control as needed
from pitaya import RedPitaya
from functiongenerator import AFG2225
from picoammeter import Keithley6487
from powersupply import NGE100
import redpitaya_mca as mca

In [6]:
nge = NGE100('USB0::0x0AAD::0x0197::5601.1414k03-100771::INSTR')
pico = Keithley6487('ASRL9::INSTR')
func = AFG2225('ASRL19::INSTR')

VisaIOError: VI_ERROR_RSRC_BUSY (-1073807246): The resource is valid, but VISA cannot currently access it.

In [ ]:
## turn on 24 Volts to power the cremat pre-amplifier
output_channel = 1 ## channel to turn on
output_voltage = 24 # Volts
nge.output(ch=output_channel, state=False)
nge.voltage(ch=output_channel, volt=output_voltage)
nge.output(ch=output_channel, state=True)

In [ ]:
# set the function generator to trigger the LED
synth_channel = 2
pulse_frequency = LED_flashing_rate ## Hz, rate of pulses
pulse_amplitude = 2.0 ## V, voltage for driving LED
amplitude_offset = pulse_amplitude/2.0 #pulse_amplitude/2 ## V, offset to give positive going pulse

func.set_frequency(synth_channel, pulse_frequency)
func.set_amplitude(synth_channel, pulse_amplitude)
func.set_offset(synth_channel, amplitude_offset)
func.set_output_load(synth_channel, "HZ") ## make sure the channel is set for high impedance
func.set_output_onoff(synth_channel, 1) ## turn on channel

True

In [ ]:
## setup the Keithley and turn it on to the breakdown voltage so it starts ramping up

keithley_time_delay = 5 ## seconds, time to wait for the keithley to stabilize at a given voltage

if(instr_to_sweep == 'keith'):
    pico.voltage_source_state(0) ## turn off source
    pico.voltage_source_range(breakdown_voltage+np.max(ov_list)) ## set the range to accomodate the highest voltage needed
    pico.voltage(breakdown_voltage) ## set to breakdown
    pico.voltage_source_state(1) ## turn on source
    time.sleep(keithley_time_delay) ## wait for the voltage to ramp up
pico.voltage(30) ## set to breakdown

In [ ]:
pico.voltage(0) ## set to breakdown

In [ ]:
## setup the Red Pitaya for waveform acquisition and for the MCA

## parameters for scope acquisition (via SCPI)
rp_size = 2048 ## number of samples to acquire
rp_delay = 8192-rp_size/2. ## trigger delay relative to start of the waveform

In [ ]:
## helper function to take waveform data

def get_waveform_data(rp, chan, num_wfs, trig='ext', trig_params = {'ch': 2, 'level': 20, 'delay': rp_delay}):
    
    wvf_array=[]
    for wf_idx in range(num_wfs):
        rp.configure(dec_fac=64)
        rp.start()
        
        if trig == 'ext':
            rp.ext_trigger()
        elif trig in [1,2]:
            rp.trigger(**trig_params)
        else:
            print("Trigger must be '''ext''' or 1,2 for channel")
            return None

        t, data = rp.acquire(ch=chan, size=rp_size, ascii=True)
        wvf_array.append(data)

    return np.array(wvf_array), t

!!!! Before running this cell make sure the SCPI server is running !!!

In [6]:
## loop over overvoltages, and set the keithley. then take scope and MCA data as desired

red = RedPitaya('172.28.175.57', username='root', password='root')

for ov in ov_list:

    ## set keithley to desired voltage (assumes wired reverse biased, so positive voltages):
    if(instr_to_sweep == 'keith'):
        pico.voltage(breakdown_voltage + ov) 
        time.sleep(keithley_time_delay) ## wait for voltage to ramp up
    elif(instr_to_sweep == 'synth'):
        func.set_amplitude(synth_channel, ov)
    else:
        print("Warning, not sweeping -- set instr_to_sweep to '''keith''' or '''synth''' ")
    
    for file_idx in range(num_scope_files):
        wf_dict = {}
        for curr_chan in wf_chan_list:
            wf_data, t = get_waveform_data(red, curr_chan, num_scope_traces)    
            wf_dict[curr_chan] = wf_data
        red.save(t, wf_dict, tag='wavefms_keith_{:.2f}OV_{}'.format(ov,file_idx), path=path)

    if(make_plots):
        plt.figure()
        for curr_chan in wf_chan_list:
            plt.plot(t, wf_dict[curr_chan][0,:])
        plt.show()   

NameError: name 'func' is not defined

New test functions to get digital data from MCA code before filled into histogram

In [ ]:
red_mca = mca.mca()
ssh = red_mca.ssh_connect()
ssh.exec_command('reboot')

In [12]:
import sys 
sys.path.append("../../RedPitaya")
import redpitaya_mca as mca
import importlib 
import matplotlib
importlib.reload(mca)

<module 'redpitaya_mca' from '../../RedPitaya\\redpitaya_mca.py'>

In [13]:
red_mca = mca.mca()
ssh = red_mca.ssh_connect()

In [14]:
length = 65536
nbytes = 2
chan = 1
dec = 4
freq = 125e6/dec
sample = 1/freq 
for x in range(10):
    #dred_mca.command(2,chan)
    red_mca.config_scope(dec=dec, chan=chan, trig_chan=chan, trig_slope=0, trig_mode=0, trig_level=10000)
    red_mca.acq_scope(dec=dec, chan=chan, trig_chan=chan, trig_slope=0, trig_mode=0, trig_level=10000, samples_pre=5000, samples_total=length, wait=1)
    for y in range(1):
        data = red_mca._socket.recv(length*nbytes*2)
        converted_data = []
        for i in range(int(len(data)/nbytes)):
            converted_data.append(int.from_bytes(data[(i*nbytes):((i+1)*nbytes)], byteorder='little', signed=True))
        print(len(converted_data))
        length = int(len(converted_data[0::2]))
        plt.plot(np.linspace(0,length*sample,length)*1e6, converted_data[0::2], zorder=10)
plt.xlabel("Time [$\mu$s]")
plt.ylabel('Amplitude [ADC Units]')
plt.show() 

6237


NameError: name 'plt' is not defined

In [ ]:
red_mca.stop_mca()
red_mca.close()

!!!! Before running this section, make sure to start the MCA app via the web interface !!!

In [ ]:
def setup_mca_data(channel_to_use):

    rp = mca.mca()

    rp.setup_mca(chan=channel_to_use, baseline_level=2**16-1000, dec=4)
    rp.reset_histo(chan=channel_to_use)
    rp.start_histo(chan=channel_to_use)

    return rp

In [ ]:
num_mca_events = 10000
int_time = int(num_mca_events/LED_flashing_rate)

for ov in ov_list[:1]:

    ## set keithley to desired voltage (assumes wired reverse biased, so positive voltages):
    if(instr_to_sweep == 'keith'):
        pico.voltage(breakdown_voltage + ov) 
        time.sleep(keithley_time_delay) ## wait for voltage to ramp up
    elif(instr_to_sweep == 'synth'):
        func.set_amplitude(synth_channel, ov)
    else:
        print("Warning, not sweeping -- set instr_to_sweep to '''keith''' or '''synth''' ")

    red_mca = setup_mca_data(channel_to_use=red_pitaya_channel-1) # 0 indexing for the channel, unlike above
    time.sleep(1) ## wait for setup to be ready -- otherwise it will read before buffer is full

    if(make_plots):
        fig=plt.figure()
        fig.patch.set_facecolor('white')
        hfig = display.display(fig, display_id=True)

    for i in range(int_time):
        dat = red_mca.read_histo_data(chan=1)

        if(make_plots):
            plt.clf()
            plt.yscale('log')
            plt.plot(dat, color='k')
            plt.ylim(0.1,10000)
            hfig.update(fig)
            time.sleep(1.0)

    red_mca.save(dat, ch=red_pitaya_channel, tag='mca_keith_{:.2f}OV'.format(ov), path=path)

    red_mca.close()